In [1]:
import torch
import numpy as np
import pickle
#from torchsummary import summary
from collections import OrderedDict
#from torch.utils.tensorboard import SummaryWriter
import datetime
import copy

In [2]:
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers)-1
        #print(layers)
        # set up layer order dict
        self.activation = torch.nn.ReLU
        
        layer_list = list()
        for i in range(self.depth-1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        #print(layers[-2])
        #print(layers[-1])
        
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [116]:
## Asssume the dim of Traing and Testing are in shape [N,C,H,W]

class DE_MLP():
    def __init__(self, outdim=1,maxdepth=70,mindepth=5,minneuron=4,maxneuron=10,bsize=10,epoch=100,initSize=20,maxiter=2000,stopcount=3,\
                 trainingset=None,validationset=None,trainingTarget=None,validateTarget=None):
        self.best=[]
        self.mean=[]
        self.outdim=outdim
        self.maxdepth=maxdepth
        self.mindepth=mindepth
        self.minneuron = minneuron
        self.maxneuron = maxneuron
        self.bszie = bsize
        self.epoch = epoch
        self.stopcount = stopcount
        self.pplSize = initSize
        self.maxiter = maxiter
        self.training = trainingset.reshape((trainingset.shape[0],-1))
        self.validationSet = validationset.reshape((validationset.shape[0],-1))
        self.target=trainingTarget
        self.validationTarget = validateTarget
        self.MLPlayerlist = []
        self.depthlist=np.random.choice(range(self.mindepth,self.maxdepth),self.pplSize,replace=False)
        
        if torch.cuda.is_available():
            self.device = torch.device('cuda')
        else:
            self.device = torch.device('cpu')

        for i in range(self.pplSize):
            depth = self.depthlist[i]
            tmp=[]
            tmp.append(self.training.shape[1])
            for j in range(depth):
                tmp.append(np.random.choice(range(self.minneuron,self.maxneuron),1,replace=False)[0])
            tmp.append(self.outdim)
            tmp=np.array(tmp)
            self.MLPlayerlist.append(tmp)
        
    def fit(config,id_):
        dnn = DNN(config)
        dnn.layers.to(self.device)
        best = float('inf')
        stop=0
        self.optimizer(dnn.layers.parameters(), lr=0.001)
        loss = torch.nn.BCEWithLogitsLoss()
        batch = self.trainingset.shape[0]//self.bsize
        vbatch = self.validationSet.shape[0]//self.bsize
        idxs = [x for x in range(self.trainingset.shape[0])]
        vidxs = [x for x in range(self.validationSet.shape[0])]
        for e in range(self.epoch):
            np.random.shuffle(idxs)
            dnn.layers.train()
            batchloss=0
            for i in range(batch):
                idx=idxs[i*self.bsize:i*self.bsize+self.bsize]
                self.optimizer.zero_grad()
                data = torch.tensor(self.trainingset[idx]).float().to(self.device)
                y = torch.tensor(self.target[idx])
                yhat = dnn(data)
                l = loss(yhat,y)
                batchloss+=l.item()
                loss.backward()
                self.optimizer.step()

            dnn.layers.eval()
            np.random.shuffle(vidxs)
            vloss=0
            for i in range(vbatch):
                vidx=vidxs[i*self.bsize:i*self.bsize+self.bsize]
                vdata = torch.tensor(self.validationSet[vidx]).float().to(self.device)
                vy = torch.tensor(self.target[vidx])
                vyhat = dnn(vdata)
                vl = loss(vyhat,vy)
                vloss += vl.item()
            if(vloss<best):
                vloss=best
            else:
                stop+=1
            print(f'ConfigID: {id_:3d}, Epoch: {e:3d}, Training Loss: {(batchloss/batch):10.8f}, Validation Loss: {(vloss/vbatch):10.8f},\
                    Best: {best:10.8f}, StopCount/Limit: {stop:3d}/{stopcount:3d}')

            if(stop>=self.stopcount):
                return best,config,id_ 

    def mutation_1_2_z(self,x1,xs,beta,debug=False):
        indim = x1[0]
        x1 = x1[1:-1] # remove in/out dim
        xs[0] = xs[0][1:-1]
        xs[1] = xs[1][1:-1]
        if(debug):
            print(f'x1 len {x1.shape[0]} xs0 len {xs[0].shape[0]} xs1 len {xs[1].shape[0]}')
        #length mutation
        minlen = np.min([x1.shape[0],xs[0].shape[0],xs[1].shape[0]])
        if(debug):
            print(f'minlen {minlen}')          
        newminlen = minlen
        targetlen=int(np.floor((x1.shape[0]) + beta * (xs[0].shape[0] - xs[1].shape[0])))
        if(targetlen==0):
            targetlen=x1.shape[0]
        elif(targetlen<0):
            targetlen=abs(targetlen)
        if(targetlen < minlen):
            newminlen=targetlen
        if(debug):
            print(f'New Min Len :{newminlen}, Length Mutation :{targetlen}')
        #node mutation
        xa = np.zeros((targetlen),dtype=int)
        xa = x1[:newminlen] + beta * (xs[0][:newminlen] - xs[1][:newminlen]) # mutate on node with minlen
        if(targetlen>minlen):
            xaa = np.zeros((targetlen-minlen))
            #mutation on the rest
            a=None
            b=None
            c=None
            for i in range(targetlen-newminlen):
                if(x1.shape[0]<=newminlen+i):
                    a=np.random.choice(range(self.mindepth,self.maxdepth),1,replace=False)
                if(x1.shape[0]>newminlen+i):
                    a=x1[newminlen+i] 

                if(xs[0].shape[0]<=newminlen+i):
                    b=np.random.choice(range(self.mindepth,self.maxdepth),1,replace=False)
                elif(xs[0].shape[0]>newminlen+i):
                    b=xs[0][newminlen+i]  

                if(xs[1].shape[0]<=newminlen+i):
                    c=np.random.choice(range(self.mindepth,self.maxdepth),1,replace=False)
                elif(xs[1].shape[0]>newminlen+i):
                    c=xs[1][newminlen+i]  
            
                xaa[i]=a + beta * (b - c)
            
            xa = np.concatenate((xa, xaa), axis=None)
            
        for i in range(xa.shape[0]):
            if(xa[i]>self.maxdepth):
                 xa[i]=self.maxdepth
            elif(xa[i]<self.mindepth):
                  xa[i]=self.mindepth
            xa[i] = np.floor(xa[i])
        xa = np.concatenate((np.array(indim,dtype=int),np.array(xa,dtype=int),np.array(self.outdim,dtype=int)), axis=None,dtype=int)
        return xa
    
    def crossover1(par):
        return np.mean(par,axis=0)
    
    def run(self,beta=0.5):
        current_gen=self.MLPlayerlist
        scores = np.zeros((self.pplSize))
        #initial Run
        #for i in len(self.MLPlayerlist):
        #    b,_,_ = self.fit(self.MLPlayerlist[i],self.stopcount,i)
        #    scores[i]=b
        
        #Generation Run
        for i in range(self.maxiter):
            currentbest = np.min(scores)
            currentmean = np.mean(scores)
            currentbestidx = np.argmin(scores)
            print(f'Run {i:5d} Best: {currentbest}, Mean: {currentmean}, ID:{currentbestidx}, config: {current_gen[currentbestidx]}')
        
            for j in range(self.pplSize):
                parent = current_gen[j]
                midxs = np.random.choice(range(0,self.pplSize),3,replace=False)
                target = self.MLPlayerlist[midxs[2]]
                unitvector = self.mutation_1_2_z(target,self.MLPlayerlist[0:2],0.5)
                print(unitvector)        

In [117]:
train=np.zeros((4,3,5,6))
val=np.zeros((4,3,5,6))
d = DE_MLP(trainingset=train,validationset=val)

In [ ]:
d.run()